Last update 11/13/2016

Generating FAST local time computations

In [1]:
import datetime as dt
import pickle
import numpy    as np
import scipy    as sp
import matplotlib.pyplot as plt

import sys
sys.path.append('c:/Users/Conrad/Documents/GitHub/PAD/')
import Convert

%matplotlib inline

In [2]:
work_path = 'C:/Users/cschiff/Documents/GitHub/FAST-and-IMAGE/'
home_path = 'C:/Users/Conrad/Documents/GitHub/FAST-and-IMAGE/'
Fast_eph_file = home_path+'FAST_def_raw.eph'
Fast_eph_fh   = open(Fast_eph_file,'r')
Fast_eph_blob = Fast_eph_fh.readlines()
Fast_eph_fh.close()

In [3]:
Fast_date_format = '%Y-%j-%H:%M:%S.%f'

In [4]:
num_records = len(Fast_eph_blob)/7
Epochs      = [0]*num_records
States      = np.zeros((num_records,6))
Ls          = np.zeros((num_records,3))
LTs         = np.zeros((num_records))
counter = 0
for i in range(num_records):
    record          = Fast_eph_blob[i*7:(i+1)*7]
    epoch_list      = record[2].split()
    epoch_str       = epoch_list[3]+'-'+epoch_list[4]+'-'+epoch_list[5]
    epoch           = dt.datetime.strptime(epoch_str,Fast_date_format)
    x,y,z,vx,vy,vz  = map(float,record[6].split()[1:7])
    state           = [x,y,z,vx,vy,vz]
    L               = [y*vz-z*vy,z*vx-x*vz,x*vy-y*vx]
    epoch_dict      = {'year':epoch.year,'month':epoch.month,'day':epoch.day,\
                       'hour':epoch.hour,'min':epoch.minute,'sec':epoch.second}
    L_SM            = Convert.convert_GCI_to_SM(epoch_dict).dot(L)
    LT_SM           = Convert.calc_LT(L_SM)    
    Epochs[counter] = epoch
    States[counter] = state
    Ls[counter]     = L
    LTs[counter]    = LT_SM
    counter         = counter + 1
Epochs = np.asarray(Epochs)

In [24]:
pickle.dump((Epochs,LTs),open('C:/Users/Conrad/Documents/GitHub/FAST-and-Image/FAST_states_pickled.p','w+'))

Spot check the results - pick a random time

In [5]:
N = 32367

In [6]:
Epochs[N]

datetime.datetime(2004, 9, 16, 14, 27, 22, 944565)

In [7]:
States[N]

array([ -8.72200871e+03,  -4.40007611e+03,   3.87816581e-04,
        -3.53086076e-01,  -9.75232749e-01,   5.77114527e+00])

In [8]:
Ls[N]

array([-25393.47805967,  50335.97916026,   6952.38291778])

In [9]:
LTs[N]

8.041178685669486

Now to estimate the local time (given that Convert has already been tested - this is just a test that Convert was applied correctly).

From basic physics, the mean local time in SM coordinates of the angular momentum vector $\vec L$ should be very close to the angle between $\vec L$ and the sun vector ${\vec r}_S$

Using a basic geometric relation of the sun's position and velocity, allows for the angular momentum to be 'quadrant checked'; basically the difference in the angles of $\vec L$ wrt ${\vec r}_S$ and ${\vec v}_S$ (sun's velocity) should be about 90 degrees and the angle between the angular momentum and the sun's velocity sets which side of noon $\vec L$ is on.

In [11]:
epoch = Epochs[N]
epoch_dict      = {'year':epoch.year,'month':epoch.month,'day':epoch.day,\
                   'hour':epoch.hour,'min':epoch.minute,'sec':epoch.second}
print epoch_dict

{'hour': 14, 'min': 27, 'month': 9, 'sec': 22, 'year': 2004, 'day': 16}


In [13]:
Sun_pos, Sun_vel = Convert.SunEph.CalcSun_Low(epoch_dict)

In [25]:
Sun_pos_norm = np.sqrt(Sun_pos.dot(Sun_pos))
Sun_vel_norm = np.sqrt(Sun_vel.dot(Sun_vel))

In [19]:
L = Ls[N]
L_norm = np.sqrt(L.dot(L))

In [26]:
angle_Sun_pos = np.arccos(L.dot(Sun_pos)/L_norm/Sun_pos_norm)*180/np.pi
angle_Sun_vel = np.arccos(L.dot(Sun_vel)/L_norm/Sun_vel_norm)*180/np.pi

In [32]:
print angle_Sun_vel - angle_Sun_pos
print angle_Sun_pos, angle_Sun_vel

87.2628935477
57.7404506336 145.003344181


In [30]:
LT_estimated = 12 - angle_Sun_pos*24/360
print LT_estimated

8.15063662442


In [31]:
LTs[N]

8.041178685669486

Success!!!